In [1]:
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, StackingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
import numpy as np

In [2]:
x, y = make_classification(n_samples=1000, n_features=20, n_classes=2, random_state=42)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

**Voting Classifier Tuning**

In [3]:
models = [
    ('lr', LogisticRegression(random_state=42)),
    ('rf', RandomForestClassifier(n_estimators=100, random_state=42)),
    ('svm', SVC(probability=True, random_state=42))
]

voting_clf = VotingClassifier(estimators=models, voting='soft')

param_grid = {
    'lr__C': [0.1, 1, 10],
    'rf__max_depth': [5, 10, None],
    'svm__C': [0.1, 1, 10]
}

grid_search = GridSearchCV(voting_clf, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(x_train, y_train)

print("Best parameters:", grid_search.best_params_)
print("Best score:", grid_search.best_score_)

Best parameters: {'lr__C': 0.1, 'rf__max_depth': None, 'svm__C': 0.1}
Best score: 0.8850000000000001


**Stacking Classifier Tuning**

In [4]:
base_models = [
    ('lr', LogisticRegression(random_state=42)),
    ('rf', RandomForestClassifier(n_estimators=100, random_state=42))
]
meta_model = LogisticRegression(random_state=42)

stacking_clf = StackingClassifier(
    estimators=base_models,
    final_estimator=meta_model,
    cv=5
)

# Tune meta-learner
param_grid = {
    'final_estimator__C': [0.1, 1, 10]
}

grid_search = GridSearchCV(
    stacking_clf, param_grid, cv=5, scoring='accuracy', n_jobs=-1
)
grid_search.fit(x_train, y_train)


print("Best parameters:", grid_search.best_params_)
print("Best score:", grid_search.best_score_)

Best parameters: {'final_estimator__C': 0.1}
Best score: 0.8899999999999999
